In [459]:
import numpy as np
import pandas as pd
from datetime import datetime

In [460]:
rd_station_profile = pd.read_csv("../data/rd_station_profile.csv")
info_student_unimestre = pd.read_csv("../data/info_student_unimestre.csv")
first_cd_turma_student = pd.read_csv("../data/first_cd_turma_for_student_unimestre.csv")
dt_cadastro_student = pd.read_csv("../data/dt_cadastro_student_unimestre.csv")

Trying to get less null values on the min_dt_log

In [461]:
info_student_unimestre.loc[:, 'min_dt_log'] = (
    info_student_unimestre['min_dt_log']
    .fillna(first_cd_turma_student['min_date'])
    .fillna(dt_cadastro_student['dt_cadastro'])
)

With this I was able to reduce 
- *min_dt_log* had 9163 null values now 209

Creating a df to be only one row per ds_email (to avoid the duplicates values checked in the exploring file)

In [462]:
info_student_unimestre_unique = info_student_unimestre.groupby('ds_email').first().reset_index()

### Doing the join again with the new df

In [463]:
merge_left = rd_station_profile.merge(info_student_unimestre_unique, left_on='email', right_on='ds_email', how='left')

In [464]:
merge_left = merge_left.drop(columns='Unnamed: 0')

Creating a new column saying it's a student when a cd_pessoa is found

In [465]:
merge_left['aluno'] = (merge_left['cd_pessoa'].notnull()).astype(int)
merge_left['aluno'].value_counts()


aluno
0    37278
1     4062
Name: count, dtype: int64

## Got different columns with representing the same thing, compiling all in one new columns

Represents city of the person: 'ds_cidade', 'Cidade Final'

In [466]:
merge_left.loc[:, 'cidade'] = (
    merge_left['ds_cidade']
    .fillna(merge_left['cidade_final'])
)

Represents city of the person: 'ds_cidade', 'Cidade Final'

In [467]:
merge_left.loc[:, 'data_nascimento'] = (
    merge_left['dt_nascimento']
    .fillna(merge_left['data_de_nascimento'])
)

With this I was able to reduce 
- *Cidade* had 35856 null values now 33986
- *Data Nascimento* had 38346 null values now 36117

Dropping columns with replicate values

In [468]:
merge_left = merge_left.drop(columns=['ds_cidade', 'cidade_final', 'dt_nascimento', 'data_de_nascimento'])

All not null

In [469]:
merge_left[merge_left.notna().all(axis=1)]

,email,lead_scoring_-_perfil,url_pública,estágio_no_funil,total_de_conversões,lead_scoring_-_interesse,status_para_comunicação_por_email,data_da_primeira_conversão,eventos_(últimos_100),origem_da_última_conversão,...,qual_o_curso_de_interesse?,cargo_final,area_atuacao,interesse_final,ds_email,cd_pessoa,min_dt_log,aluno,cidade,data_nascimento
5431,carolineklock@hotmail.com,c,http://app.rdstation.com.br/leads/public/8717b...,Lead Qualificado,16,120,True,2015-09-01 19:17:37 -0300,jornada-assessment-instrumentos-para-lideranca...,Busca Orgânica | Bing,...,Educação empresarial / Remuneração,Outros Cargos,RH,formulario-de-pre-inscricao,carolineklock@hotmail.com,128609.0,2018-04-05 19:53:30,1,Joinville,1985-09-21 00:00:00
15881,stella.bousfield@gmail.com,c,http://app.rdstation.com.br/leads/public/1b271...,Cliente,27,170,True,2015-12-08 10:18:55 -0200,webinar-storytelling / indicado-sarau-de-negoc...,Desconhecido,...,Responsabilidade social,Outros Cargos,Marketing,webinar-storytelling,stella.bousfield@gmail.com,160000017.0,2020-02-27 17:28:20,1,Joinville,1977-06-02 00:00:00


### Change dtype for columns that are date

In [470]:
def convert_columns_to_datetime(df):
    """
    Convert specified columns in a DataFrame to datetime format.

    Parameters:
    - df: DataFrame
        The input DataFrame.
    - date_columns: list
        List of column names to convert to datetime.

    Returns:
    - DataFrame
        The DataFrame with specified columns converted to datetime.
    """

    date_columns = ['data_da_primeira_conversão', 'data_da_última_conversão', 'data_da_última_oportunidade', 'data_nascimento', 'min_dt_log']
    
    df[date_columns] = df[date_columns].apply(lambda col: pd.to_datetime(col, errors='coerce'))
    for i in date_columns:
        df[i] = pd.to_datetime(df[i], utc=True).dt.tz_convert(None)
    return df

merge_left = convert_columns_to_datetime(merge_left)
merge_left.dtypes


C:\Users\anafl\AppData\Local\Temp\ipykernel_45952\3683617211.py:18: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df[date_columns] = df[date_columns].apply(lambda col: pd.to_datetime(col, errors='coerce'))
C:\Users\anafl\AppData\Local\Temp\ipykernel_45952\3683617211.py:18: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df[date_columns] = df[date_columns].apply(lambda col: pd.to_datetime(col, errors='coerce'))
C:\Users\anafl\AppData\Local\Temp\

email                                                                        object
lead_scoring_-_perfil                                                        object
url_pública                                                                  object
estágio_no_funil                                                             object
total_de_conversões                                                           int64
lead_scoring_-_interesse                                                      int64
status_para_comunicação_por_email                                              bool
data_da_primeira_conversão                                           datetime64[ns]
eventos_(últimos_100)                                                        object
origem_da_última_conversão                                                   object
data_da_última_conversão                                             datetime64[ns]
origem_da_primeira_conversão                                                

In [471]:
def calculate_months_since_conversion(df):
    """
    Calculates the number of months since the first conversion based on the 'aluno' column and additional date columns.
    Drops rows where 'aluno' = 1 and min_dt_log is null.

    Parameters:
    - df: The input DataFrame containing the necessary columns.
    
    Returns:
    - df: The DataFrame with additional columns representing the number of months since the first conversion,
          last conversion, and last opportunity. Null values are assigned to these columns if the corresponding dates are null.
    """
    # Drop rows based on conditions
    df = df.drop(df[(df['aluno'] == 1) & df['min_dt_log'].isnull()].index)

    # Calculate months since conversion
    df['months_since_first_conversion'] = np.where(
        df['aluno'] == 1,
        (df['min_dt_log'] - df['data_da_primeira_conversão']).dt.days / 30,
        (datetime.now() - df['data_da_primeira_conversão']).dt.days / 30
    )

    # Calculate months since last conversion
    df['months_since_last_conversion'] = np.where(
        df['data_da_última_conversão'].notnull(),
        (df['data_da_última_conversão'] - df['data_da_primeira_conversão']).dt.days / 30,
        np.nan
    )

    # Calculate months since last opportunity
    df['months_since_last_opportunity'] = np.where(
        df['data_da_última_oportunidade'].notnull(),
        (df['data_da_última_oportunidade'] - df['data_da_primeira_conversão']).dt.days / 30,
        np.nan
    )

    #df = df.drop(columns=['data_da_primeira_conversão', 'min_dt_log', 'data_da_última_conversão', 'data_da_última_oportunidade'])

    return df

# Apply the combined function to the DataFrame
perfil_df = calculate_months_since_conversion(merge_left)


In [472]:
perfil_df.columns

Index(['email', 'lead_scoring_-_perfil', 'url_pública', 'estágio_no_funil',
       'total_de_conversões', 'lead_scoring_-_interesse',
       'status_para_comunicação_por_email', 'data_da_primeira_conversão',
       'eventos_(últimos_100)', 'origem_da_última_conversão',
       'data_da_última_conversão', 'origem_da_primeira_conversão', 'empresa',
       'tags', 'data_da_última_oportunidade', 'estado',
       'valor_total_da_oportunidade_no_crm_(última_atualização)',
       'qualificação_da_oportunidade_no_crm_(última_atualização)',
       'etapa_do_funil_de_vendas_no_crm_(última_atualização)',
       'nome_do_responsável_pela_oportunidade_no_crm_(última_atualização)',
       'origem_da_oportunidade_no_crm_(última_atualização)',
       'possui_interesse_em_realizar_uma_pós-graduação_ou_mba?',
       'desejo_receber_o_sustentare_news', 'como_você_conheceu_a_sustentare:',
       'universidade', 'qual_o_curso_de_interesse?', 'cargo_final',
       'area_atuacao', 'interesse_final', 'ds_email

In [473]:
perfil_df.sample()

,email,lead_scoring_-_perfil,url_pública,estágio_no_funil,total_de_conversões,lead_scoring_-_interesse,status_para_comunicação_por_email,data_da_primeira_conversão,eventos_(últimos_100),origem_da_última_conversão,...,interesse_final,ds_email,cd_pessoa,min_dt_log,aluno,cidade,data_nascimento,months_since_first_conversion,months_since_last_conversion,months_since_last_opportunity
17619,tonyrepresentacao@gmail.com,d,http://app.rdstation.com.br/leads/public/f4b10...,Lead,1,0,True,2020-05-04 22:25:57,Participante Curso Conta Azul - Empréstimos e ...,Desconhecido,...,NaN,NaN,NaN,NaT,0,NaN,NaT,43.5,0.0,NaN


In [474]:
perfil_df.columns

Index(['email', 'lead_scoring_-_perfil', 'url_pública', 'estágio_no_funil',
       'total_de_conversões', 'lead_scoring_-_interesse',
       'status_para_comunicação_por_email', 'data_da_primeira_conversão',
       'eventos_(últimos_100)', 'origem_da_última_conversão',
       'data_da_última_conversão', 'origem_da_primeira_conversão', 'empresa',
       'tags', 'data_da_última_oportunidade', 'estado',
       'valor_total_da_oportunidade_no_crm_(última_atualização)',
       'qualificação_da_oportunidade_no_crm_(última_atualização)',
       'etapa_do_funil_de_vendas_no_crm_(última_atualização)',
       'nome_do_responsável_pela_oportunidade_no_crm_(última_atualização)',
       'origem_da_oportunidade_no_crm_(última_atualização)',
       'possui_interesse_em_realizar_uma_pós-graduação_ou_mba?',
       'desejo_receber_o_sustentare_news', 'como_você_conheceu_a_sustentare:',
       'universidade', 'qual_o_curso_de_interesse?', 'cargo_final',
       'area_atuacao', 'interesse_final', 'ds_email

## Creating a Function for the Age

In [475]:
def create_column_with_calculate_age(df):
    """
    Calculates age based on the birthdate column and adds a new column to the DataFrame.

    Parameters:
    - df: The input DataFrame.

    Returns:
    - df: The DataFrame with the new age column added.
    """
    df['age'] = pd.to_datetime(df['data_nascimento'], errors='coerce').dt.year
    current_year = pd.to_datetime('now').year
    df['age'] = current_year - df['age']

    # Set age to NaN for values less than 15 or greater than 100
    df.loc[(df['age'] < 15) | (df['age'] > 100), 'age'] = pd.NA
    
    #df = df.drop(columns='data_nascimento')

    return df

perfil_df = create_column_with_calculate_age(perfil_df)


In [476]:
perfil_df.sample()

,email,lead_scoring_-_perfil,url_pública,estágio_no_funil,total_de_conversões,lead_scoring_-_interesse,status_para_comunicação_por_email,data_da_primeira_conversão,eventos_(últimos_100),origem_da_última_conversão,...,ds_email,cd_pessoa,min_dt_log,aluno,cidade,data_nascimento,months_since_first_conversion,months_since_last_conversion,months_since_last_opportunity,age
3257,patricia.chiele@monacobr.com.br,c,http://app.rdstation.com.br/leads/public/c5544...,Lead,1,0,True,2023-05-23 11:10:23,inscricao-festival-rh-agil-3-edicao,Email | festival_rh_agil_-_3_edicao_,...,NaN,NaN,NaT,0,NaN,NaT,6.4,0.0,NaN,NaN


In [477]:
perfil_df.to_csv('../data/clear_perfil_visitor_and_students.csv')

# Extraction tableau

In [478]:
perfil_df[perfil_df['aluno'] == 1]['lead_scoring_-_interesse'].value_counts()



lead_scoring_-_interesse
0       1221
10       589
50       281
60       224
20       181
70       135
30        94
80        83
100       54
90        51
40        49
95        28
110       27
120       24
85        24
75        22
35        18
130       17
25        16
115       15
45        13
105       12
55        12
150       11
160       11
140       10
125        8
190        7
145        6
65         5
205        4
170        4
155        4
200        4
175        3
135        3
250        2
235        2
245        2
225        2
165        2
300        2
255        2
230        2
550        1
860        1
1005       1
500        1
215        1
275        1
180        1
195        1
395        1
325        1
185        1
Name: count, dtype: int64